In [1]:
using JuMP

In [2]:
# Define model
m = Model()

# Road/Snow removal machine combinations
S = ["11","12","21","22","31","32"]

# Non-negativity
@variable(m, x[S] >= 0)

# Objective function: maximize snow removal
@objective(m, Max, x["11"] + 2x["12"] + 1.2x["21"] + 3x["22"] + 2x["31"] + 3x["32"])

# Each machine can only operate for 24 hours in a day
@constraint(m, x["11"] + x["21"] + x["31"] <= 24)
@constraint(m, x["12"] + x["22"] + x["32"] <= 24)

# Can only plow existing snow from each road
@constraint(m, x["11"] + 2x["12"] <= 15)
@constraint(m, 1.2x["21"] + 3x["22"] <= 24)
@constraint(m, 2x["31"] + 3x["32"] <= 18)

# Solve the optimazation problem
solve(m)

# Determine road/snow machine combinations
println("Variable values: ", getvalue(x))

# Determine optimal amount of snow reamoved
println("Objective value: ", getobjectivevalue(m))

Variable values: x: 1 dimensions:
[11] = 0.0
[12] = 7.5
[21] = 0.0
[22] = 8.0
[31] = 9.0
[32] = 0.0

Objective value: 57.0


In [3]:
# Define model
m2 = Model()

# Road/Snow removal machine combinations
S = ["11","12","21","22","31","32"]

# Non-negativity
@variable(m2, x[S] >= 0)
@variable(m2, z >= 0)

# Minimize snow removal time
@objective(m2, Min, z)

# Each machine can only be on the road the min amount of time
@constraint(m2, x["11"] + x["21"] + x["31"] <= z)
@constraint(m2, x["12"] + x["22"] + x["32"] <= z)

# All the snow must be removed from each road
@constraint(m2, x["11"] + 2x["12"] == 15)
@constraint(m2, 1.2x["21"] + 3x["22"] == 24)
@constraint(m2, 2x["31"] + 3x["32"] == 18)

# Solve the optimazation problem
solve(m2)

# Determine road/snow machine combinations
println("Variable values: ", getvalue(x))

# Determine optimal amount of snow removed
println("Objective value: ", getobjectivevalue(m2))



Variable values: x: 1 dimensions:
[11] = 4.333333333333333
[12] = 5.333333333333334
[21] = 0.0
[22] = 8.0
[31] = 9.0
[32] = 0.0

Objective value: 13.333333333333332


In [54]:
# Define model
diet = Model()

# Non-negativity and define food variables
@variable(diet, x >= 0)
@variable(diet, y >= 0)

# Objective function: minimize cost
@objective(diet, Min, 2x + 3y)

# Set up constraints
# Need to obtain minimum amount of nutrients
# Nutrient 1
@constraint(diet, 4x + 6y >= 30)
# Nutrient 2
@constraint(diet, 6x + 2y >= 20)
# Nutrient 3
@constraint(diet, x + 2y >= 12)

# Solve the optimazation problem
status = solve(diet)

# Determine food amount
println("Variable value x: ", getvalue(x))
println("Variable value y: ", getvalue(y))

# Determine optimal cost of food
println("Objective value: ", getobjectivevalue(diet))
# What is the total quantity of food produced
println("Total amount: ", getvalue(x) + getvalue(y))

Variable value x: 1.5999999999999996
Variable value y: 5.2
Objective value: 18.8
Total amount: 6.8


In [113]:
# Import dataframes package and read in teh fooddata csv
using DataFrames
fooddata = readtable("fooddata.csv",skipstart=1,nrows=10,eltypes=[UTF8String,Int64,Int64,Int64,Int64,Int64])
println(fooddata)

10×6 DataFrames.DataFrame
│ Row │ Food │ Calories │ Fat │ Saturated_Fat │ Carbohydrate │ Protein │
├─────┼──────┼──────────┼─────┼───────────────┼──────────────┼─────────┤
│ 1   │ "1"  │ 448      │ 1   │ 5             │ 36           │ 2       │
│ 2   │ "2"  │ 305      │ 17  │ 4             │ 40           │ 7       │
│ 3   │ "3"  │ 337      │ 4   │ 5             │ 22           │ 4       │
│ 4   │ "4"  │ 455      │ 18  │ 2             │ 83           │ 5       │
│ 5   │ "5"  │ 314      │ 19  │ 1             │ 76           │ 13      │
│ 6   │ "6"  │ 437      │ 19  │ 5             │ 45           │ 1       │
│ 7   │ "7"  │ 438      │ 3   │ 1             │ 93           │ 0       │
│ 8   │ "8"  │ 413      │ 17  │ 3             │ 77           │ 0       │
│ 9   │ "9"  │ 481      │ 13  │ 5             │ 54           │ 0       │
│ 10  │ "10" │ 150      │ 14  │ 0             │ 22           │ 2       │


In [111]:
# Define model
food = Model()

# Food options
F = fooddata[:Food]

# Non-negativity and define food variables
@variable(food, x[F] >= 0)

# Objective function: 
@objective(food, Min, sum([fooddata[i,:Calories]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]))





448 x[1] + 305 x[2] + 337 x[3] + 455 x[4] + 314 x[5] + 437 x[6] + 438 x[7] + 413 x[8] + 481 x[9] + 150 x[10]

10×6 DataFrames.DataFrame
│ Row │ Food │ Calories │ Fat │ Saturated_Fat │ Carbohydrate │ Protein │
├─────┼──────┼──────────┼─────┼───────────────┼──────────────┼─────────┤
│ 1   │ "1"  │ 448      │ 1   │ 5             │ 36           │ 2       │
│ 2   │ "2"  │ 305      │ 17  │ 4             │ 40           │ 7       │
│ 3   │ "3"  │ 337      │ 4   │ 5             │ 22           │ 4       │
│ 4   │ "4"  │ 455      │ 18  │ 2             │ 83           │ 5       │
│ 5   │ "5"  │ 314      │ 19  │ 1             │ 76           │ 13      │
│ 6   │ "6"  │ 437      │ 19  │ 5             │ 45           │ 1       │
│ 7   │ "7"  │ 438      │ 3   │ 1             │ 93           │ 0       │
│ 8   │ "8"  │ 413      │ 17  │ 3             │ 77           │ 0       │
│ 9   │ "9"  │ 481      │ 13  │ 5             │ 54           │ 0       │
│ 10  │ "10" │ 150      │ 14  │ 0             │ 22           │ 2       │
